# GPT-2

## Loading libraries

In [15]:
!pip install fire

  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=d9e1d8f04cfba7eba7e4494878185e318bb10ec52328b098154c923907d61807
  Stored in directory: c:\users\thinkpad\appdata\local\pip\cache\wheels\5b\eb\43\7295e71293b218ddfd627f935229bf54af9018add7fbb5aac6
Successfully built fire


In [1]:
#load libraries to code gpt-2 from scratch
import numpy as np

## implementing the GPT-2 model

In [2]:
def gelu(x):
    return 0.5 * x * (1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * x**3)))

In [3]:
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

In [4]:
def layer_norm(x, g, b, eps: float = 1e-5):
    mean = np.mean(x, axis=-1, keepdims=True)
    variance = np.var(x, axis=-1, keepdims=True)
    x = (x - mean) / np.sqrt(variance + eps)


In [5]:
def linear(x, w, b): 
    return x @ w + b

In [6]:
def ffn(x, c_fc, c_proj):
    a = gelu(linear(x, **c_fc))
    x = linear(a, **c_proj)
    return x

In [8]:
def attention(q, k, v, mask): 
    """
    q: [n_q, d_q] the query
    k: [n_k, d_k] the key
    v: [n_v, d_v] the value
    mask: [n_q, n_k] the mask to prevent the model from attending to the future tokens
    
    the query q, key k, and value v are all matrices of shape [n, d] where n is the number of tokens and d is the dimension of the token
    the mask is a matrix of shape [n_q, n_k] where n_q is the number of queries and n_k is the number of keys
    the mask is used to prevent the model from attending to the future tokens
    the utility of the query is that it can be used to select the most relevant key-value pairs from the key-value pairs
    the utility of the key is that it can be used to select the most relevant query from the queries
    """
    attention_score = q @ k.T # [n_q, d_k], [n_k, d_k] -> [n_q, n_k] the dot product of q and k is the attention score
    attention_score = attention_score / np.sqrt(q.shape[-1]) # scale the attention score by the square root of the dimension of q to prevent the score from exploding
    attention_score = attention_score + mask # add the mask to the attention score to prevent the model from attending to the future tokens
    attention_weights = softmax(attention_score) # apply softmax to the attention score to get the attention weights
    return attention_weights @ v # [n_q, n_k], [n_k, d_v] -> [n_q, d_v] the dot product of the attention weights and v is the output of the attention layer

    


In [9]:
def mha(x, c_attn, c_proj, n_head):  # [n_seq, n_embd] -> [n_seq, n_embd]
    # qkv projection
    x = linear(x, **c_attn)  # [n_seq, n_embd] -> [n_seq, 3*n_embd]

    # split into qkv
    qkv = np.split(x, 3, axis=-1)  # [n_seq, 3*n_embd] -> [3, n_seq, n_embd]

    # split into heads
    qkv_heads = list(map(lambda x: np.split(x, n_head, axis=-1), qkv))  # [3, n_seq, n_embd] -> [3, n_head, n_seq, n_embd/n_head]

    # causal mask to hide future inputs from being attended to
    causal_mask = (1 - np.tri(x.shape[0], dtype=x.dtype)) * -1e10  # [n_seq, n_seq]

    # perform attention over each head
    out_heads = [attention(q, k, v, causal_mask) for q, k, v in zip(*qkv_heads)]  # [3, n_head, n_seq, n_embd/n_head] -> [n_head, n_seq, n_embd/n_head]

    # merge heads
    x = np.hstack(out_heads)  # [n_head, n_seq, n_embd/n_head] -> [n_seq, n_embd]

    # out projection
    x = linear(x, **c_proj)  # [n_seq, n_embd] -> [n_seq, n_embd]

    return x

In [10]:
def transformer_block(x, mlp, attn, ln_1, ln_2, n_head):  # [n_seq, n_embd] -> [n_seq, n_embd]
    # multi-head causal self attention
    x = x + mha(layer_norm(x, **ln_1), **attn, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]

    # position-wise feed forward network
    x = x + ffn(layer_norm(x, **ln_2), **mlp)  # [n_seq, n_embd] -> [n_seq, n_embd]

    return x

In [11]:
def gpt2(inputs, wte, wpe, blocks, ln_f, n_head):  # [n_seq] -> [n_seq, n_vocab]
    # token + positional embeddings
    x = wte[inputs] + wpe[range(len(inputs))]  # [n_seq] -> [n_seq, n_embd]

    # forward pass through n_layer transformer blocks
    for block in blocks:
        x = transformer_block(x, **block, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]

    # projection to vocab
    x = layer_norm(x, **ln_f)  # [n_seq, n_embd] -> [n_seq, n_embd]
    return x @ wte.T  # [n_seq, n_embd] -> [n_seq, n_vocab]

In [12]:

def gpt2(inputs, wte, wpe, blocks, ln_f, n_head):  # [n_seq] -> [n_seq, n_vocab]
    # token + positional embeddings
    x = wte[inputs] + wpe[range(len(inputs))]  # [n_seq] -> [n_seq, n_embd]

    # forward pass through n_layer transformer blocks
    for block in blocks:
        x = transformer_block(x, **block, n_head=n_head)  # [n_seq, n_embd] -> [n_seq, n_embd]

    # projection to vocab
    x = layer_norm(x, **ln_f)  # [n_seq, n_embd] -> [n_seq, n_embd]
    return x @ wte.T  # [n_seq, n_embd] -> [n_seq, n_vocab]

In [13]:
def generate(inputs, params, n_head, n_tokens_to_generate):
    from tqdm import tqdm

    for _ in tqdm(range(n_tokens_to_generate), "generating"):  # auto-regressive decode loop
        logits = gpt2(inputs, **params, n_head=n_head)  # model forward pass
        next_id = np.argmax(logits[-1])  # greedy sampling
        inputs.append(int(next_id))  # append prediction to input

    return inputs[len(inputs) - n_tokens_to_generate :]  # only return generated ids


In [17]:
def main(prompt: str, n_tokens_to_generate: int = 40, model_size: str = "124M", models_dir: str = "models"):
    from utils import load_encoder_hparams_and_params

    # load encoder, hparams, and params from the released open-ai gpt-2 files
    encoder, hparams, params = load_encoder_hparams_and_params(model_size, models_dir)

    # encode the input string using the BPE tokenizer
    input_ids = encoder.encode(prompt)

    # make sure we are not surpassing the max sequence length of our model
    assert len(input_ids) + n_tokens_to_generate < hparams["n_ctx"]

    # generate output ids
    output_ids = generate(input_ids, params, hparams["n_head"], n_tokens_to_generate)

    # decode the ids back into a string
    output_text = encoder.decode(output_ids)

    return output_text


if __name__ == "__main__":
    import fire

    fire.Fire(main)

ERROR: The function received no value for the required argument: prompt
Usage: ipykernel_launcher.py PROMPT <flags>
  optional flags:        --n_tokens_to_generate | --model_size | --models_dir

For detailed information on this command, run:
  ipykernel_launcher.py --help


FireExit: 2